In [6]:
"""
Projeto: Limpador de e-mails

Este projeto faz uma conexão a uma conta de e-mail, utilizando bibliotecas Python,
com autenticação através de senha de aplicativo gerada pelo Gmail.

O objetivo era aplicar na prática os conhecimentos adquiridos durante o curso, como:
- manipulação de strings,
- criação e uso de funções,
- estruturas condicionais (if-else),
- laços de repetição (for),
entre outros.

Projeto feito totalmente para fins educativos afim de fixar o conteúdo.
"""
import imaplib
import ssl
import email
from email.header import decode_header
from datetime import datetime
from getpass import getpass
import re

email_usuario = input("Digite seu e-mail: ")
senha = getpass("Digite sua senha: ")


palavras = input("Digite as palavras-chave para filtrar (separadas por vírgula): ")
palavras_chave = [a.strip() for a in palavras.split(",")]

remetentes = input("Digite os remetentes que deseja filtrar (separados por vírgula): ")
recebidos = [b.strip() for b in remetentes.split(",")]


def login():
    contexto = ssl.create_default_context()
    servidor = imaplib.IMAP4_SSL("imap.gmail.com", 993, ssl_context=contexto)
    servidor.login(email_usuario, senha)
    print("\033[32mConexão realizada com sucesso.\033[0m")
    return servidor

servidor = login()

def ler(servidor):
    servidor.select("inbox")
    status, mensagens = servidor.search(None, "ALL")
    ids = mensagens[0].split()

    print(f"\nTotal de e-mails encontrados: {len(ids)}")


def filtro(assunto, remetente):
    for palavra in palavras_chave:
        if palavra.lower() in assunto.lower():
            return True
    for filtro in recebidos:
        if filtro.lower() in remetente.lower():
            return True
    return False


def deletar(servidor, limite=50):
    servidor.select("inbox")
    status, mensagens = servidor.search(None, "ALL")
    ids = mensagens[0].split()

    emails_deletados = []

    for num in ids[-limite:][::-1]:
        status, dados = servidor.fetch(num, "(RFC822)")
        conteudo = dados[0][1]
        msg = email.message_from_bytes(conteudo)

        assunto = msg["Subject"] or ""
        if assunto:
            assunto_decodificado, encoding = decode_header(assunto)[0]
            if isinstance(assunto_decodificado, bytes):
                assunto = assunto_decodificado.decode(encoding or "utf-8", errors="ignore")

        remetente = msg.get("From") or ""
        data = msg.get("Date")
        try:
            data_formatada = email.utils.parsedate_to_datetime(data).strftime("%d/%m/%Y")
        except:
            data_formatada = data

        if filtro(assunto, remetente):
            print(f"\033[31m{assunto} | {remetente} | {data_formatada}\033[0m")
            emails_deletados.append(num)

    if emails_deletados:
        confirmar = input(f"\n{len(emails_deletados)} e-mail(s) correspondem aos filtros. \nDeseja deletar esses e-mails? (s/n): ").strip().lower()
        if confirmar == "s":
            for num in emails_deletados:
                servidor.store(num, "+FLAGS", "\\Deleted")
                servidor.expunge()
            print(f"\033[32m{len(emails_deletados)} e-mails excluidos.\033[0m")
        else:
            print("Operação cancelada.")
    else:
        print("\nNenhum e-mail correspondeu aos filtros.")

ler(servidor)
deletar(servidor, limite=50)

Digite seu e-mail: gih.alciati@gmail.com
Digite sua senha: ··········
Digite as palavras-chave para filtrar (separadas por vírgula): off, inglês, duolingo
Digite os remetentes que deseja filtrar (separados por vírgula): info@mail.shopee.com.br
Conexão realizada com sucesso.

Total de e-mails encontrados: 12187

E-mails encontrados: 12187

Sua fatura SParcelado já está disponível | Shopee <info@mail.shopee.com.br> | 20/07/2025 14:03

1 e-mail(s) correspondem aos filtros. 
Deseja deletar esses e-mails? (s/n): s
1 e-mails excluidos.
